# ProPyCore SDK Examples: Submittals

This notebook contains snippets from the `submittals` module.

In [1]:
import os
import dotenv
import json
import csv

import ProPyCore

## Setup
You will need to create the connection to your Procore app and then get the details for your company and project.

### Connection to Procore App
Ensure you have a `.env` file with the following information included:
* `CLIENT_ID`: your data connection app's client ID
* `CLIENT_SECRET`: your data connection app's client secret

In [2]:
dotenv.load_dotenv()
connection = ProPyCore.procore.Procore(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    redirect_uri="urn:ietf:wg:oauth:2.0:oob", # default for data connection apps
    oauth_url="https://app.procore.com", # default for data connection apps
    base_url="https://app.procore.com" # default for data connection apps
)

### Get Company Details
Use the cell below to specify your company name

In [3]:
companies = connection.companies.get()
for company in companies:
    print(company["name"])

company_name = companies[0]["name"]

Rogers-O`Brien Construction


Now get the company details

In [4]:
COMPANY = connection.companies.find(identifier=company_name)
projects = connection.projects.get(company_id=COMPANY["id"])
PROJECT = connection.projects.find(company_id=COMPANY["id"], identifier="OpX Test Project")

### `submittals.get_statuses()`
```python
"""
Gets all the available submittal statuses

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project

Returns
-------
submittal_statuses : dict
    available submittal statuses
"""
```

In [6]:
statuses = connection.submittals.get_statuses(
    company_id=COMPANY["id"],
    project_id=PROJECT["id"],
)

print(json.dumps(statuses, indent=4))

[
    {
        "key": 2141,
        "value": "Draft"
    },
    {
        "key": 1,
        "value": "Open"
    },
    {
        "key": 115339,
        "value": "Approved / No Exceptions Taken"
    },
    {
        "key": 4095,
        "value": "Revise and Resubmit"
    },
    {
        "key": 4098,
        "value": "Void"
    }
]


### `submittals.get_types()`
```python
"""
Gets all the available submittal types

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project

Returns
-------
submittal_types : dict
    available submittal types
"""
```

In [7]:
statuses = connection.submittals.get_types(
    company_id=COMPANY["id"],
    project_id=PROJECT["id"],
)

print(json.dumps(statuses, indent=4))

[
    {
        "key": "Attic Stock",
        "value": "Attic Stock"
    },
    {
        "key": "BIM",
        "value": "BIM"
    },
    {
        "key": "Calculations",
        "value": "Calculations"
    },
    {
        "key": "Certificates / Certifications",
        "value": "Certificates / Certifications"
    },
    {
        "key": "LEED Data",
        "value": "LEED Data"
    },
    {
        "key": "Mock-up",
        "value": "Mock-up"
    },
    {
        "key": "O&M Manual",
        "value": "O&M Manual"
    },
    {
        "key": "O&M Training",
        "value": "O&M Training"
    },
    {
        "key": "Other",
        "value": "Other"
    },
    {
        "key": "Product Data",
        "value": "Product Data"
    },
    {
        "key": "Record Submittal - As-Builts",
        "value": "Record Submittal - As-Builts"
    },
    {
        "key": "Record Submittal - Photos",
        "value": "Record Submittal - Photos"
    },
    {
        "key": "Record Submittal - Reports

### `submittals.get()`
```python
"""
Gets all the available submittals

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project
page : int, default 1
    page number
per_page : int, default 100
    number of companies to include

Returns
-------
submittals : dict
    available rfi data
"""
```

In [ ]:
submittals = connection.submittals.get(
    company_id=COMPANY["id"],
    project_id=2563870,
    per_page=10000
)

print(f"Total Submittals: {len(submittals)}")
print(f"Submittal {submittals[0]['id']}: {submittals[0]['title']}")
print(json.dumps(submittals[0], indent=4))

with open("submittals.json", "w") as f:
    json.dump(submittals, f, indent=2)

Total Submittals: 690
Submittal 59439747: Evaporation Retardant and Finishing Aid (PD)
{
    "id": 59439747,
    "actual_delivery_date": null,
    "approvers": [
        {
            "id": 147669060,
            "approver_type": "Approver",
            "associated_attachments": [
                {
                    "id": 134115274,
                    "additional_page_count": 0,
                    "attached_at": "2025-02-05T14:09:31Z",
                    "attachment_id": 5219862258,
                    "content_type": "application/pdf",
                    "document_markup_layer_id": 58977669,
                    "filename": "03 05 10.0 - Concrete Moisture Evaporation Retardant Aid_Structures Reviewed.pdf",
                    "markup_updated_at": null,
                    "state": "current",
                    "updated_at": "2025-02-05T14:09:31Z",
                    "url": "https://storage.procore.com/api/v5/files/us-east-1/pro-core.com/companies/9808/01JKB71ZYVQQ8FYPF7EKW47XA1

#### Filter by Status

In [7]:
statuses = connection.submittals.get_statuses(
    company_id=COMPANY["id"],
    project_id=PROJECT["id"],
)

for status in statuses:
    filtered_submittals = connection.submittals.get(
        company_id=COMPANY["id"],
        project_id=PROJECT["id"],
        per_page=10000,
        status_ids=status["key"]
    )

    print(f"Number of {status['value']} submittals: {len(filtered_submittals)}")

Number of Draft submittals: 27
Number of Open submittals: 2
Number of Approved / No Exceptions Taken submittals: 1
Number of Approved as Noted submittals: 1
Number of Revise and Resubmit submittals: 1
Number of Submitted for Approval submittals: 1
Number of Void submittals: 1


#### Filter by Type

In [26]:
types_to_find = [
    "Product Data & Shop Drawings",
    "Product Information",
    "Product Manual",
    "Submittal",
    "Sustainable Design Submittal",
    "Test Report",
    "Certification",
    "Closeout",
    "Default for SBMTL",
    "Document",
    "Manufacturer's Specs",
    "Mix Design",
    "Mock-up",
    "O&M Manual",
    "O&M Training",
    "Other",
    "Plans",
    "Prints"
]

results = []

#projects = connection.projects.get(company_id=COMPANY["id"])
try:
    for project in projects[300:]: # 149: 252013 - ECX Austin DFW33220N PH1
        project_number = project["project_number"]
        project_name = project["name"]
        print(f"{project_number} - {project_name}")
        for type_to_find in types_to_find:
            submittals_by_type = connection.submittals.get(
                company_id=COMPANY["id"],
                project_id=PROJECT["id"],
                types=type_to_find
            )
            if len(submittals_by_type) > 0:
                print(f"\t{type_to_find}: {len(submittals_by_type)}")
                for submittal in submittals_by_type:
                    results.append({
                        "project_number": project_number,
                        "project_name": project_name,
                        "submittal_number": submittal["number"],
                        "submittal_name": submittal["title"],
                        "submittal_link": f"https://app.procore.com/webclients/host/companies/8089/projects/{project['id']}/tools/submittals/{submittal['id']}"
                    })
                    #print(f"Added {len(results)} submittals of type {type_to_find} for {project_number}: {project_name}")
except Exception as e:
    print(f"Error: {e}")

if len(results) > 0:
    with open('submittals_by_type.csv', 'w', newline='') as csvfile:
        fieldnames = ['project_number', 'project_name', 'submittal_number', 'submittal_name', 'submittal_link']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow(result)

Test 1234 - New Test Project
213002 - Niagara - Conroe
211069 - Niagara - Dallas
232009 - Niagara Temple Storage Facility
212066 - Niagra Temple FS Conversion
244002 - North Central Baptist Hybrid Renovation
194506 - North Rim Corporate Campus
201101 - Novel Turtle Creek Apartment
221014 - Oakhouse Apartments
221005 - Old Denton and Jackson Retail
192517 - OLD VRBO
192519 - One West Hills
245006 - Oppidan Temple
000000.3 - OpX Test Project
211006 - PCBC Children's Renovation P2
221102 - PCBC Columbarium
201115 - PCBC Pleitz 1st Flr Renovation
231032 - PCBC Sanctuary
251007 - PCBC Sanctuary Fire Protection
251007 - PCBC Sanctuary Fire Protection
234007 - Perlen House
182505 - Perry Mansion Hotel
191613 - Pinkston HS
162516 - Plaza Saltillo
PS212.088 - Precon 212088 - Red Bluff Office Bldg
000000.5 - Precon Test Project
231020 - Prime DFW01-01
231066 - Prime DFW01-02
None - Procore Connect Test Project
391001 - Procore Test Project 3
211050 - Project Pike - Niagara Bottling
None - Punch 

### `submittals.find()`
```python
"""
Finds specified submittal.

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project
identifier : int or str
    identifier for Submittal

Returns
-------
submittal_info : dict
    submittal data
"""
```

In [8]:
# Find by ID
submittal1 = connection.submittals.find(
    company_id=company["id"],
    project_id=projects[0]["id"],
    identifier=32747369
)

print(submittal1["title"])

# Find by Title
submittal2 = connection.submittals.find(
    company_id=company["id"],
    project_id=projects[0]["id"],
    identifier="Glass Entrances and Storefront Door Hardware - Schedule"
)
print(submittal2["id"])
print(json.dumps(submittal2,indent=4))

Glass Entrances and Storefront Door Hardware - Schedule


KeyboardInterrupt: 

---